# Benchmarking Our Algorithm against others

In [5]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


from panav.sequential import sequential_planning
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [7]:
bloating_r = 0.5
vmax = 1.0

TIMEOUT = 100

# Specify the Algorithms
algs = {  
        "Seq_Tube": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        "Seq_Tube_nonLazy": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),      
        "Seq_S2M2": lambda env: sequential_planning(Simple_MILP_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),
        "ST-RRT*": lambda env: sequential_ST_RRTStar(env,vmax,bloating_r)
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [8]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
max_N = 40
N_range = np.arange(1,max_N+1)

In [15]:
# results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Env: MultiTunnel Alg: Seq_Tube N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 9/11
Planning for agent 10/11
On time for algorithm Seq_Tube N agent =  11
Env: MultiTunnel Alg: Seq_Tube_nonLazy N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 9/11
Planning for agent 10/11
On time for algorithm Seq_Tube_nonLazy N agent =  11
Env: MultiTunnel Alg: Seq_S2M2 N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 2/11Info:    SpaceTimeRRT: Created 76 states (34 start + 42 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 3/11
Info:    SpaceTimeRRT: Created 63 states (25 start + 38 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 4/11
Info:    SpaceTimeRRT: Created 46 states (23 start + 23 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 5/11
Info:    SpaceTimeRRT: Created 35 states (19 start + 16 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Planning for agent 26/30Info:    SpaceTimeRRT: Created 604 states (237 start + 367 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Planning for agent 27/30Info:    SpaceTimeRRT: Created 553 states (225 start + 328 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Algorithm timeout
Timeout
Time out for algorithm ST-RRT* N agent =  30


In [16]:
env_name = 'SingleTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
max_N = 40
N_range = np.arange(1,max_N+1)

In [17]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Env: SingleTunnel Alg: Seq_Tube N agent: 1
Planning for agent 0/1
On time for algorithm Seq_Tube N agent =  1
Env: SingleTunnel Alg: Seq_Tube_nonLazy N agent: 1
Planning for agent 0/1
On time for algorithm Seq_Tube_nonLazy N agent =  1
Env: SingleTunnel Alg: Seq_S2M2 N agent: 1
Planning for agent 0/1
On time for algorithm Seq_S2M2 N agent =  1
Env: SingleTunnel Alg: ST-RRT* N agent: 1
Planning for agent 0/1
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Created 155 states (61 start + 94 goal)
On time for algorithm ST-RRT* N agent =  1
Env: SingleTunnel Alg: Seq_Tube N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube N agent =  2
Env: SingleTunnel Alg: Seq_Tube_nonLazy N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube_nonLazy N agent =  2
Env: SingleTunnel Alg: Seq_S2M2 N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_S2M2 N agent =  2
Env: SingleTunnel Alg: ST-RRT* N agent: 2
Planning for agent 0/2
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 1/2
Info:    SpaceTimeRRT: Created 86 states (39 start + 47 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in da

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


On time for algorithmInfo:    SpaceTimeRRT: Created 231 states (103 start + 128 goal)
 ST-RRT* N agent =  2
Env: SingleTunnel Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Env: SingleTunnel Alg: Seq_Tube_nonLazy N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube_nonLazy N agent =  3
Env: SingleTunnel Alg: Seq_S2M2 N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_S2M2 N agent =  3
Env: SingleTunnel Alg: ST-RRT* N agent: 3
Planning for agent 0/3
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Planning for agent 16/18Info:    SpaceTimeRRT: Created 971 states (400 start + 571 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Info:    SpaceTimeRRT: Increased time bound factor to 8.00
Planning for agent 17/18Info:    SpaceTimeRRT: Created 1589 states (724 start + 865 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Algorithm timeout
Timeout
Time out for algorithm ST-RRT* N agent =  18
Env: SingleTunnel Alg: Seq_Tube N agent: 19
Planning for agent 0/19


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 1/19
Planning for agent 2/19
Planning for agent 3/19
Planning for agent 4/19
Planning for agent 5/19
Planning for agent 6/19
Planning for agent 7/19
Planning for agent 8/19
Planning for agent 9/19
Planning for agent 10/19
MILP low-level encounters infeasibility.
On time for algorithm Seq_Tube N agent =  19
Env: SingleTunnel Alg: Seq_Tube N agent: 20
Planning for agent 0/20
Planning for agent 1/20
Planning for agent 2/20
Planning for agent 3/20
Planning for agent 4/20
Planning for agent 5/20
Planning for agent 6/20
Planning for agent 7/20
Planning for agent 8/20
Planning for agent 9/20
Planning for agent 10/20
Planning for agent 11/20
Planning for agent 12/20
Planning for agent 13/20
Planning for agent 14/20
Planning for agent 15/20
Planning for agent 16/20
Planning for agent 17/20
Planning for agent 18/20
Planning for agent 19/20
On time for algorithm Seq_Tube N agent =  20
Env: SingleTunnel Alg: Seq_Tube N agent: 21
Planning for agent 0/21
Planning for agent 1/21
Pl